<a href="https://colab.research.google.com/github/Ragavan7071/ece-cit/blob/master/poetry_generation_with_telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import time


In [2]:
test='/content/drive/My Drive/test.txt'
text1 = open(test,'rb').read().decode(encoding='cp1251')
text=text1.replace("<eos>","\n")
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1944400 characters


In [3]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

62 unique characters


In [4]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [5]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '$' :   4,
  '%' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  ',' :  10,
  '.' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '4' :  16,
  '5' :  17,
  '6' :  18,
  '7' :  19,
  ...
}


In [6]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

w
i
t
h
 


In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'with clytia he no longer was received \n than while he was a man of wealth believed \n balls , concerts'
" , op'ras , tournaments , and plays \n expensive dresses , all engaging ways \n were used to captivate "
'this lady fair \n while scarcely one around but in despair \n wife , widow , maid , his fond affection '
"sought \n to gain him , ev'ry wily art was brought \n but all in vain :пїЅ by passion overpow'red \n the"
' belle , whose conduct others would have soured \n to him appeared a goddess full of charms \n superior'


In [8]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text
dataset = sequences.map(split_input_target)

In [9]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'with clytia he no longer was received \n than while he was a man of wealth believed \n balls , concert'
Target data: 'ith clytia he no longer was received \n than while he was a man of wealth believed \n balls , concerts'


In [10]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 53 ('w')
  expected output: 39 ('i')
Step    1
  input: 39 ('i')
  expected output: 50 ('t')
Step    2
  input: 50 ('t')
  expected output: 38 ('h')
Step    3
  input: 38 ('h')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 33 ('c')


In [11]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [14]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [15]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 62)


In [16]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [17]:
print("Input:", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 'eed \n the poetry of earth is ceasing never \n on a lone winter evening , when the frost \n has wrought'

Next Char Predictions: 
 '\n<6v6q52;4p.sb\'v!(tq%b" i"<z>o>!k(e$[)ax8g)?et$ho 8l6l.%[})п>\nї"y64w %.?t26ioi%;("70!{(oпnїvx0xmlmg$'


In [18]:
!git clone https://github.com/Ragavan7071/ece-cit.git

Cloning into 'ece-cit'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 153 (delta 24), reused 0 (delta 0), pack-reused 114
Receiving objects: 100% (153/153), 27.77 MiB | 14.79 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [19]:
tf.train.latest_checkpoint('/content/drive/My Drive/training_checkpoints')

'/content/drive/My Drive/training_checkpoints/ckpt_30'

In [21]:
checkpoint_dir='/content/drive/My Drive/training_checkpoints'

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [23]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 670
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.2

  model.reset_states()
  for i in range(num_generate):
    # using a categorical distribution to predict the character returned by the model
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [29]:
slash=input("Enter the starting string:")
slash=slash.lower()
print("\n The poem is:\n")
s=generate_text(model, start_string=slash)

Enter the starting string:bhasker

 The poem is:



In [25]:
!pip install gTTS

  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4096 sha256=3e9def85d2fd192d98fb218138f74862a8d925ad9e9a07f4875ea1e14381989c
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token


In [30]:
print(s)

bhasker silver steeds 
 all in their common watery rob snows 
 o glar 
 to sport my corder , dome did ye could view thy winds grown clear 
 and always this ? i say othe sons 
 in soverarcian colonsation is to view 
 the only boots are lost them all his god 
 comander , what match thou might'st thy fum пїЅ re shed 
 taste the belecate and so pums up in human shore 
 viiling young idom bounds 
 all vapour is the pipe clown at , пїЅ mid their heads confrized most over books of carpuse threw the tides grow 
 under 
 the nimble fairies by the socarain i loint to prayer 
 his thunder round in hiscent , and s l'tched wires diet , hopeless , and come frowned 
 avail for juddme


In [31]:
from gtts import gTTS
from IPython.display import Audio
t=gTTS(s)
t.save('s.wav')
a='s.wav'
Audio(a, autoplay=True)

In [ ]:
#secondary API interfacing
#code to access telegram 

import json 
import requests
#here token refers to your bot API token
TOKEN = "1225137884:AAH_8CYyi4RP_tiNf9pKSpbZjy9qAAJRqzE"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)


def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates():
    url = URL + "getUpdates"
    js = get_json_from_url(url)
    return js


def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return (text, chat_id)


def send_message(text, chat_id):
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)
   


In [ ]:
 text,chat=get_last_chat_id_and_text(get_updates())
(text,'r')
text.replace('/','')
text=text.lower()
print('the poem for the given string is:\n')
out=generate_text(model,start_string=text)
print(out)
send_message(out,chat)